In [ ]:
# Install package to read parquet files
!pip3 install pyarrow

In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
all_years = [2021]
all_months = ['09', '10', '11', '12']
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/'
file_name = 'yellow_tripdata_%d-%s.parquet'

In [3]:
for year in all_years:
    for month in all_months:
        yellow_csv = file_name % (year, month)
        response = requests.get( url + yellow_csv, allow_redirects=True)
        open( '../data/raw/'+yellow_csv, 'wb').write(response.content)

FileNotFoundError: [Errno 2] No such file or directory: 'data/curated/yellow_tripdata_2021-09.parquet'

In [ ]:
all_yellow_trip_data = pd.DataFrame()

for year in all_years:
    for month in all_months:
        yellow_csv = pd.read_parquet( file_name % (year, month) )
        all_yellow_trip_data = all_yellow_trip_data.append('../data/raw/'+yellow_csv)

In [ ]:
all_yellow_trip_data.info()

In [ ]:
# Missing data
import numpy as np

features_nan = [feature for feature in all_yellow_trip_data.columns if all_yellow_trip_data[feature].isnull().sum() > 1]
for feature in features_nan:
    print(f"{feature}: {np.round(all_yellow_trip_data[feature].isnull().mean(), 4)}% missing values")

In [ ]:
# Clean the empty data
all_data_nonan = all_yellow_trip_data.dropna()
all_data_nonan.isnull().sum()

In [ ]:
# Look at the unique values for different columns
for column in all_data_nonan.columns:
    series = all_data_nonan[column]
    print('Feature : ', column , ' min : ', series.min(), ' max : ', series.max())

In [ ]:
cleaned_data = all_data_nonan[all_data_nonan["passenger_count"] > 0]
cleaned_data  = cleaned_data [cleaned_data["trip_distance"] != 0]
cleaned_data  = cleaned_data [(cleaned_data ['tpep_pickup_datetime']<='2021-12-31 23:59:59') & (cleaned_data ['tpep_pickup_datetime']>='2021-09-01 00:00:00')]
cleaned_data  = cleaned_data [(cleaned_data ['tpep_dropoff_datetime']<='2021-12-31 23:59:59') & (cleaned_data ['tpep_dropoff_datetime']>='2021-09-01 00:00:00')]

cleaned_data = cleaned_data[cleaned_data["payment_type"] == 1]
cleaned_data = cleaned_data[cleaned_data["fare_amount"] >= 2.5]

In [ ]:
check_feature = ['fare_amount','tip_amount','total_amount']
import matplotlib.pyplot as plt
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(9,9))
#fig.suptitle("Outliers of amounts before cleaning", fontsize=30, y=0.95)
plt.subplots_adjust(hspace=0.5)
i = 0
for k in check_feature:
    cleaned_data.plot(y=k, kind='box', ax=axs.ravel()[i])
    i += 1
plt.show()

In [ ]:
# Clean the outlier in amount feature
amount = ["fare_amount", "tip_amount", "total_amount"]
Q1 = cleaned_data[amount].quantile(0.25)
Q3 = cleaned_data[amount].quantile(0.75)
IQR = Q3 - Q1
cleaned = cleaned_data[~((cleaned_data[amount] < (Q1-3*IQR)) | (cleaned_data[amount] > (Q3+3*IQR))).any(axis=1)]


In [ ]:
# Plot result
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(9, 9))
#fig.suptitle("Outliers of amounts after cleaning", fontsize=30, y=0.95)
plt.subplots_adjust(hspace=0.5)
i = 0
for k in amount:
    cleaned.plot(y=k, kind='box', ax=axs.ravel()[i])
    i += 1
plt.show()

In [ ]:
# Process time information into new features

cleaned['month'] = pd.to_datetime(cleaned['tpep_pickup_datetime']).dt.month
cleaned['day'] = pd.to_datetime(cleaned['tpep_pickup_datetime']).dt.day
cleaned['week'] = pd.to_datetime(cleaned['tpep_pickup_datetime']).dt.week
cleaned['weekday'] = pd.to_datetime(cleaned['tpep_pickup_datetime']).dt.weekday
cleaned['starting_hour'] = pd.to_datetime(cleaned['tpep_pickup_datetime']).dt.hour
cleaned['ending_hour'] = pd.to_datetime(cleaned['tpep_dropoff_datetime']).dt.hour
cleaned['trip_time'] = (pd.to_datetime(cleaned['tpep_dropoff_datetime'])-pd.to_datetime(cleaned['tpep_pickup_datetime'])).dt.total_seconds()
cleaned['date'] = cleaned['tpep_pickup_datetime'].apply(str).str[:13]

In [ ]:
cleaned.head(20)

In [ ]:
cleaned_data.to_csv('../data/curated/data_processed.csv')

In [ ]:
# Load weather data
weather_data = pd.read_csv("../data/raw/weather_hourly.csv")
weather_data.info()

In [ ]:
# Pick features
weather_data_pick = weather_data[['datetime','feelslike','humidity','windspeed','visibility','solarradiation','conditions']]
weather_data_pick['datetime'] = weather_data_pick['datetime'].str[:13]
weather_data_pick['datetime'] = weather_data_pick['datetime'].str.replace('T',' ')

weather_data_pick = weather_data_pick.join(pd.get_dummies(weather_data_pick.conditions,prefix='conditions'))
#weather_data_pick = weather_data_pick.drop('conditions',axis=1)
weather_data_pick.head(10)

In [ ]:
weather_data_pick.to_csv('../data/curated/processed_weather.csv')

In [ ]:
for column in weather_data_pick.columns:
    series =  weather_data_pick[column]
    print('Feature : ', column , ' min : ', series.min(), ' max : ', series.max())

In [ ]:
# Merge to get new data
data = pd.merge(cleaned, weather_data_pick, left_on='date', right_on='datetime')

In [ ]:
data = data.drop(['datetime','date'],axis=1)
data.head(50)

In [ ]:
data.to_csv('../data/curated/processed.csv') # Save processe data to save time